In [1]:
import cv2, os, random, pickle
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image, ImageDraw

In [2]:
def getCountyBBOX(image):
    # https://stackoverflow.com/questions/4808221/is-there-a-bounding-box-function-slice-with-non-zero-values-for-a-ndarray-in
    B = np.argwhere(image)
    (ystart, xstart), (ystop, xstop) = B.min(0), B.max(0) + 1 
    return [xstart, ystart, xstop, ystop]

In [3]:
base_dir = r"C:\Users\franc\OneDrive - University Of Houston\AAA_RECTDNN\data\BBNN\scrap\\"
out_dir = r"C:\Users\franc\OneDrive - University Of Houston\AAA_RECTDNN\data\BBNN\BBNN_OriginalData\\"

base_basedir = f"{base_dir}in\\"
tile_basedir = f"{base_dir}tiles\\"
cnty_basedir = f"{base_dir}county\\"

image_dir = f"{out_dir}bbox_analysis\\"
pkl_dir   = f"{out_dir}pkl\\"
    
def exportImage(image, predictions, image_path):
    image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR))
    # Draw bounding boxes on the image
    draw = ImageDraw.Draw(image)
    for box, label in zip(predictions['boxes'], predictions['labels']):
        draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline='red')
        draw.text((box[0], box[1]), f'Label: {label}', fill='red')

    # Save the image with bounding boxes
    image.save(image_path[:-3]+"png")

for name in tqdm(os.listdir(tile_basedir)):
    
    # READ AS NEEDED
    image_or_tiles = cv2.imread(os.path.join(tile_basedir, name))
    image_or_cnty  = cv2.imread(os.path.join(cnty_basedir, name))
    if image_or_cnty is None:
        image_or_cnty  = cv2.imread(os.path.join(cnty_basedir, name[:-3] + "tif"))
    
    # CONVERT TO GRAYSCALE (MAYBE NEEDED)
    image_tile = cv2.cvtColor(image_or_tiles, cv2.COLOR_BGR2GRAY)
    image_cnty = cv2.cvtColor(image_or_cnty, cv2.COLOR_BGR2GRAY)
    
    contours, hierarchy = cv2.findContours(image_tile, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    county_bbox = getCountyBBOX(image_cnty > 0)
    
    # 0 for county
    # 1 for tile
    
    out_dict = {
        'boxes'  : [county_bbox],
        'labels' : ["County"] 
    }

    highest_level = np.max(hierarchy, axis=1).flatten()[3]
    for idx, contour in enumerate(contours):
        if hierarchy[0][idx][3] == highest_level: 
            x, y, w, h = cv2.boundingRect(contour)
            out_dict['boxes'].append([x, y, x+w, y+h])
            out_dict['labels'].append("Tile")
            
    exportImage(image_tile, out_dict, image_dir+name)
    
    with open(pkl_dir+name[:-3]+'pkl', 'wb') as handle:
        pickle.dump(out_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    # print(out_dict)

  0%|          | 0/14 [00:00<?, ?it/s]